In [11]:
import pandas as pd
import matplotlib
import star_players


def classify_player(row):
    if row['display_first_last'] in (star_players.get_star_players()):
        return 'STAR'
    elif row['to_year'] - row['from_year'] >= 5:
        return 'SOLID'
    else:
        return 'BUST'

college_df = pd.read_csv("college-stats.csv", low_memory=False)

college_df = college_df.loc[college_df.groupby('player_name')['year'].idxmax()]

nba_df = pd.read_csv("common_player_info.csv")
draft_year = 2009

nba_df = nba_df[nba_df["from_year"] >= draft_year]

nba_df.sort_values(by=['from_year'], inplace=True)

nba_df['classification'] = nba_df.apply(classify_player, axis=1)

# We will remove the college players that most recently entered the NBA, as we do not have enough information to determine
# whether they are a bust or a solid player yet.
nba_df = nba_df[
        ~(
            (nba_df['from_year'] >= 2019) &
            (nba_df['to_year'] >= 2023) &
            (nba_df['classification'] == 'BUST')
        )
    ]


# dataframe = dataframe[(dataframe['classification'] != 'BUST') & (dataframe['to_year'] - dataframe['from_year'] < 5)]

columns = ['display_first_last', 'classification', 'from_year', 'to_year', 'school']

nba_df = nba_df[columns]

# now to add data to the dataframe freom other files.
nba_df.to_csv('output.csv', index=False)


## THIS IS WHERE WE ARE MERGING THE FILES

classified__nba_df = pd.read_csv("output.csv")


combined_df = classified__nba_df.merge(college_df, left_on=['display_first_last', 'from_year'], right_on=['player_name', 'year'], how='inner')
draft_combine = pd.read_csv("draft_combine_stats.csv")
combined_df = combined_df.merge(draft_combine, left_on='display_first_last', right_on='player_name', how='inner')
# best sample I can get of college data


combined_df = combined_df[star_players.get_training_columns()]
combined_df.to_csv('output.csv', index=False)

# print(combined_df['classification'].value_counts()['BUST'])
# print(combined_df['classification'].value_counts()['SOLID'])
# print(combined_df['classification'].value_counts()['STAR'])


We will now look at a visualization that shows the distribution of the three different classifications of NBA players, "stars",
"solid" players, and "busts

In [16]:
classifications = combined_df.groupby(by='classification').size()
display(classifications)

# plot = classifications.plot.bar(color=['red','blue','green'],rot=0)

# we will look at a visualization that shows the amount of




classification
BUST     151
SOLID    143
STAR      31
dtype: int64